In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

In [2]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject = '01',
                             session = '01',
                             task = 'audiovisual',
                             run = '01',
                             datatype = 'meg',
                             root = bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq = 0.1, h_freq = 40)
events, event_id = mne.events_from_annotations(raw)

Opening raw data file out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading events from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv.
Reading channel info from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


C:\Users\hp\AppData\Roaming\Python\Python37\site-packages\mne_bids\read.py:406: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw.set_channel_types(channel_type_dict)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 19821 samples (33.001 sec)

Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Left', 'Visual/Right']


In [3]:
event_id

{'Auditory/Left': 1,
 'Auditory/Right': 2,
 'Button': 3,
 'Smiley': 4,
 'Visual/Left': 5,
 'Visual/Right': 6}